In [13]:
import sys
sys.path.insert(0, "../src")

import re
import os
import xgboost as xgb
import datetime
import numpy as np
import torch as th
import pandas as pd
import pickle as pkl
from utils import log
from sklearn.metrics import roc_auc_score


def get_date(url):
    dates = re.findall(r"\d\d\d\d\/\d\d\/\d\d", url)
    return next(iter(dates), None)


In [2]:
with open("../data/prepared.pkl", "rb") as fp:
    prepared = pkl.load(fp)
vocabulary = prepared["vocabulary"]
texts = prepared["texts"]
contexts = prepared["contexts"]
test_texts = prepared["test_texts"]
y_train = prepared["y_train"]
y_test = prepared["y_test"]
text_train = prepared["texts_train"]
text_test = prepared["texts_test"]


In [3]:
x_train = np.load(
    # "../data/xmlr_train.npy"
    "../data/xmlr_train_2.npy"
)
# x_test = np.load("../data/xmlr_embs_test.npy")


In [4]:
x_train.shape, y_train.shape

((63356, 768), (63356,))

In [7]:
xgb.XGBClassifier?

Init signature:
xgb.XGBClassifier(
    *,
    objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'binary:logistic',
    **kwargs: Any,
) -> None
Docstring:     
Implementation of the scikit-learn API for XGBoost classification.
See :doc:`/python/sklearn_estimator` for more information.

Parameters
----------

    n_estimators : Optional[int]
        Number of boosting rounds.

    max_depth :  Optional[int]
        Maximum tree depth for base learners.
    max_leaves :
        Maximum number of leaves; 0 indicates no limit.
    max_bin :
        If using histogram-based algorithm, maximum number of bins per feature
    grow_policy :
        Tree growing policy. 0: favor splitting at nodes closest to the node, i.e. grow
        depth-wise. 1: favor splitting at nodes with highest loss change.
    learning_rate : Optional[float]
        Boosting learning rate (xgb's "eta")
    verbosity : Optional[int]
        The degree of ve

In [10]:
est = xgb.XGBClassifier(verbosity=3)

In [11]:
est.fit(
    x_train[-30_000:], 
    y_train[-30_000:]
)

[17:08:42] ======== Monitor (0): HostSketchContainer ========
[17:08:42] AllReduce: 0.123985s, 1 calls @ 123985us

[17:08:42] MakeCuts: 0.130744s, 1 calls @ 130744us

[17:08:42] DEBUG: /workspace/src/gbm/gbtree.cc:130: Using tree method: 0
[17:12:32] ======== Monitor (0): Learner ========
[17:12:32] Configure: 0.000419s, 1 calls @ 419us

[17:12:32] EvalOneIter: 0.001369s, 100 calls @ 1369us

[17:12:32] GetGradient: 0.497866s, 100 calls @ 497866us

[17:12:32] PredictRaw: 0.000402s, 100 calls @ 402us

[17:12:32] UpdateOneIter: 229.219s, 100 calls @ 229219315us

[17:12:32] ======== Monitor (0): GBTree ========
[17:12:32] BoostNewTrees: 228.719s, 100 calls @ 228719233us

[17:12:32] CommitModel: 0.000261s, 100 calls @ 261us

[17:12:32] ======== Monitor (0): HistUpdater ========
[17:12:32] BuildHistogram: 84.8853s, 5944 calls @ 84885328us

[17:12:32] EvaluateSplits: 73.5745s, 7144 calls @ 73574482us

[17:12:32] InitData: 2.56608s, 1200 calls @ 2566081us

[17:12:32] InitRoot: 36.5069s, 1200 c

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [14]:
p_train = est.predict_proba(x_train)
roc_auc_score(y_true=y_train, y_score=p_train, multi_class="ovo")

0.9891047075581737

In [17]:
path = "../data/xmlr_embs_test.npy"
x_test = np.load(path)
p_test = est.predict_proba(x_test)
roc_auc_score(y_true=y_test, y_score=p_test, multi_class="ovo")

0.9605041454494184

In [20]:
import lightgbm as lgb
est = lgb.LGBMClassifier()
est.fit(x_train[-30_000:],y_train[-30_000:])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.194582 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 30000, number of used features: 768
[LightGBM] [Info] Start training from score -1.706500
[LightGBM] [Info] Start training from score -2.726214
[LightGBM] [Info] Start training from score -3.071174
[LightGBM] [Info] Start training from score -2.601440
[LightGBM] [Info] Start training from score -3.218043
[LightGBM] [Info] Start training from score -3.731091
[LightGBM] [Info] Start training from score -2.959079
[LightGBM] [Info] Start training from score -2.785471
[LightGBM] [Info] Start training from score -1.749658
[LightGBM] [Info] Start training from score -2.275943
[LightGBM] [Info] Start training from score -2.313646
[LightGBM] [Info] Start training from score -2.531998


LGBMClassifier()

In [21]:
p_train = est.predict_proba(x_train)
roc_auc_score(y_true=y_train, y_score=p_train, multi_class="ovo")

0.9885520556406182

In [22]:
path = "../data/xmlr_embs_test.npy"
x_test = np.load(path)
p_test = est.predict_proba(x_test)
roc_auc_score(y_true=y_test, y_score=p_test, multi_class="ovo")

0.9605908057371698